In [9]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Flatten 
from tensorflow.keras.layers import Conv2D 
from tensorflow.keras.layers import MaxPooling2D 
from tensorflow.keras.applications import VGG16

In [10]:
# get generators from py file 
%run image_generators.py

Found 9537 images belonging to 2 classes.
Found 1906 images belonging to 2 classes.
Found 8054 images belonging to 2 classes.
Found 1610 images belonging to 2 classes.


In [11]:

model = Sequential()
# define 3x3 filter window sizes. Create 32 filters.
model.add(Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=(224, 224, 3)))


# max pool in 2x2 window
model.add(MaxPooling2D(pool_size=(2, 2)))
# define 3x3 filter window sizes. Create 64 filters.
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))

# connect nodes all nodes with dense layers 
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='softmax'))


bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)


model.compile(optimizer='adam', loss=bce,  metrics=['accuracy'])

In [4]:
# fit model - CT

CT_base_model = model.fit(
        ct_train_generator,
        steps_per_epoch=100,verbose = 1,validation_data = ct_validation_generator, 
        epochs=10)

Epoch 1/10


2023-03-14 15:47:18.806930: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
/Users/brendanhutchinson/miniforge3/envs/brain_net/lib/python3.10/site-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


100/100 [==============================] - 52s 512ms/step - loss: 0.5008 - accuracy: 0.7744 - val_loss: 0.4590 - val_accuracy: 0.8714
Epoch 2/10
100/100 [==============================] - 52s 516ms/step - loss: 0.2871 - accuracy: 0.8621 - val_loss: 0.3948 - val_accuracy: 0.8037
Epoch 3/10
100/100 [==============================] - 53s 531ms/step - loss: 0.2641 - accuracy: 0.8765 - val_loss: 0.3164 - val_accuracy: 0.8820
Epoch 4/10
100/100 [==============================] - 53s 526ms/step - loss: 0.2299 - accuracy: 0.8969 - val_loss: 0.2714 - val_accuracy: 0.9031
Epoch 5/10
100/100 [==============================] - 53s 525ms/step - loss: 0.2157 - accuracy: 0.9119 - val_loss: 0.3631 - val_accuracy: 0.8323
Epoch 6/10
100/100 [==============================] - 54s 544ms/step - loss: 0.1943 - accuracy: 0.9141 - val_loss: 0.0756 - val_accuracy: 0.9770
Epoch 7/10
100/100 [==============================] - 53s 531ms/step - loss: 0.1814 - accuracy: 0.9247 - val_loss: 0.1380 - val_accuracy: 0.9

In [5]:
# fit model - X-Ray
XR_base_model = model.fit(
        XR_train_generator,
        steps_per_epoch=100,verbose =1,validation_data = xr_validation_generator, 
        epochs=10)

Epoch 1/10
100/100 [==============================] - 69s 686ms/step - loss: 0.5155 - accuracy: 0.7638 - val_loss: 0.4489 - val_accuracy: 0.8033
Epoch 2/10
100/100 [==============================] - 70s 699ms/step - loss: 0.3447 - accuracy: 0.8609 - val_loss: 0.3692 - val_accuracy: 0.8242
Epoch 3/10
100/100 [==============================] - 68s 679ms/step - loss: 0.3160 - accuracy: 0.8716 - val_loss: 0.3211 - val_accuracy: 0.8662
Epoch 4/10
100/100 [==============================] - 101s 1s/step - loss: 0.2664 - accuracy: 0.8937 - val_loss: 0.2651 - val_accuracy: 0.8930
Epoch 5/10
100/100 [==============================] - 68s 679ms/step - loss: 0.2335 - accuracy: 0.9091 - val_loss: 0.2460 - val_accuracy: 0.8987
Epoch 6/10
100/100 [==============================] - 68s 679ms/step - loss: 0.2146 - accuracy: 0.9178 - val_loss: 0.2269 - val_accuracy: 0.9161
Epoch 7/10
100/100 [==============================] - 70s 697ms/step - loss: 0.2242 - accuracy: 0.9166 - val_loss: 0.2209 - val_accu

In [ ]:
# base model evaluation - CT scans

y_pred = model.predict(ct_train_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(ct_train_generator, axis=1)

cm = confusion_matrix(y_true_classes, y_pred_classes)
print(cm)



y_pred = model.predict(ct_train_generator)
fpr, tpr, thresholds = roc_curve(ct_train_generator, y_pred)
roc_auc = auc(fpr, tpr)

import matplotlib.pyplot as plt

plt.plot(fpr, tpr)
         
         
         

206/252 [=======================>......] - ETA: 2:10

In [13]:
## vgg16

vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

vggmodel = tf.keras.models.Sequential()
vggmodel.add(vgg_model)
vggmodel.add(tf.keras.layers.Flatten())
vggmodel.add(tf.keras.layers.Dense(256, activation='relu'))
vggmodel.add(tf.keras.layers.Dropout(0.5))
vggmodel.add(tf.keras.layers.Dense(1, activation='softmax'))

vggmodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
vggmodel.fit_generator(ct_train_generator,
                    steps_per_epoch=20,
                    epochs=10,
                    validation_data=ct_validation_generator)

Epoch 1/10


/var/folders/nw/j3b8rwtj6057njmc_kj0q5540000gp/T/ipykernel_94049/659340891.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(ct_train_generator,


20/20 [==============================] - ETA: 0s - loss: 6.2383 - accuracy: 0.5000

KeyboardInterrupt: 

In [ ]:
vggmodel.fit_generator(xr_train_generator,
                    steps_per_epoch=20,
                    epochs=10,
                    validation_data=xr_validation_generator)

In [ ]:
# tuning base model 

model2 = Sequential()

model2.add(Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=(224, 224, 3)))



model2.add(MaxPooling2D(pool_size=(2, 2)))

model2.add(Conv2D(64, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(64, (3, 3), activation='relu'))


model2.add(Flatten())
model2.add(Dense(64, activation='relu'))
model2.add(Dense(8, activation='tahn'))
model2.add(Dense(2, activation='sigmoid'))
